In [1]:
# Import Libraries

import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target
import tensorflow as tf
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier  # Use scikeras for scikit-learn compatibility
from math import floor
from bayes_opt import BayesianOptimization
from tensorflow.keras.layers import LeakyReLU  # Use tensorflow.keras instead of keras
LeakyReLU = LeakyReLU(negative_slope=0.1)
import warnings

2024-07-20 14:39:34.018554: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
path = '/Users/mariazaremba/Documents/CareerFoundry/Machine Learning with Python'

In [3]:
climate_cleaned = pd.read_csv(os.path.join(path, 'Data', 'Prepared Data','climate_cleaned_DATE.csv'), index_col = False)

In [4]:
climate_cleaned.shape

(22950, 137)

In [5]:
climate_cleaned.head()

,Unnamed: 0,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,DATE
0,0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,...,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9,19600101
1,1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,...,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1,19600102
2,2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,...,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9,19600103
3,3,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,...,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6,19600104
4,4,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,...,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4,19600105


In [6]:
#Read in the pleasant weather data.
pleasantweather = pd.read_csv(os.path.join(path, 'Data', 'Original Data', 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

In [7]:
pleasantweather.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
pleasantweather.shape

(22950, 16)

In [9]:
climate_cleaned = climate_cleaned.drop(columns=['DATE', 'Unnamed: 0'])

In [10]:
pleasantweather = pleasantweather.drop('DATE', axis=1)

In [11]:
climate_cleaned.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4


In [12]:
climate_cleaned.shape

(22950, 135)

In [13]:
pleasantweather.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
pleasantweather.shape

(22950, 15)

In [15]:
#Create an 'X' and 'y' matrix 
X = climate_cleaned
y = pleasantweather

In [16]:
X = np.array(X)
y = np.array(y)

In [17]:
X = X.reshape(-1,15,9)

In [18]:
X.shape

(22950, 15, 9)

In [19]:
# Reshape y to (22950,)
y = np.argmax(y, axis=1)
print("y shape after argmax:", y.shape)

y shape after argmax: (22950,)


In [20]:
#Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [21]:
type_of_target(y)

'multiclass'

In [22]:
# Determine the number of time steps for the input data
timesteps = X_train.shape[1]

# Determine the dimensionality of the input data
input_dim = X_train.shape[2]

# Specify the number of classes for the target variable
n_classes = 15  

# Create a scorer for accuracy
score_acc = make_scorer(accuracy_score)

In [23]:
# Create function
def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs, layers1, layers2, normalization, dropout, dropout_rate):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'SGD']
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]
    optimizer_name = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)

    def cnn_model():
        model = Sequential()
        model.add(Input(shape=(timesteps, input_dim)))
        model.add(Conv1D(neurons, kernel_size=kernel, activation=activation))
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=127))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax'))  # sigmoid softmax
        
        # Create a new optimizer instance for each iteration
        if optimizer_name == 'Adam':
            optimizer_instance = Adam(learning_rate=learning_rate)
        elif optimizer_name == 'SGD':
            optimizer_instance = SGD(learning_rate=learning_rate)
        elif optimizer_name == 'RMSprop':
            optimizer_instance = RMSprop(learning_rate=learning_rate)
        elif optimizer_name == 'Adadelta':
            optimizer_instance = Adadelta(learning_rate=learning_rate)
        elif optimizer_name == 'Adagrad':
            optimizer_instance = Adagrad(learning_rate=learning_rate)
        elif optimizer_name == 'Adamax':
            optimizer_instance = Adamax(learning_rate=learning_rate)
        elif optimizer_name == 'Nadam':
            optimizer_instance = Nadam(learning_rate=learning_rate)
        elif optimizer_name == 'Ftrl':
            optimizer_instance = Ftrl(learning_rate=learning_rate)
        
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer_instance, metrics=['accuracy'])
        return model

    # K-fold cross-validation
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=20)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=127)
    results = []
    for train, test in kfold.split(X, y):
        model = cnn_model()
        model.fit(X[train], y[train], epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[es])
        scores = model.evaluate(X[test], y[test], verbose=1)
        results.append(scores[1])  # Assuming accuracy is the second metric
    return np.mean(results)

In [24]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000),
    'epochs':(20, 100),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) #25
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6602 - loss: 1.9732
Epoch 20: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4109 - loss: 4.6346
Epoch 20: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6403 - loss: 4.3699
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4337 - loss: 5.1350
Epoch 20: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3639 - loss: 6.6350
| 1         | 0.4903    | 3.371     | 960.6     | 0.732     | 0.1796    | 32.48     | 1.312     | 1.116     | 2.732     | 0.6051    | 73.73     | 0.02058   | 6.789     |
Epoch 21: early stopping
144/144 ━━━━━━━

144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6547 - loss: 2.3732
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6520 - loss: 41.0183
Epoch 20: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6609 - loss: 555.8156
Epoch 20: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6544 - loss: 71.4351
| 9         | 0.5854    | 4.705     | 542.0     | 0.02542   | 0.03237   | 22.51     | 2.273     | 1.629     | 2.017     | 0.9085    | 32.44     | 0.4104    | 5.289     |
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7085 - loss: 0.8675
Epoch 20: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6918 - loss: 0.9172
Epoch 20: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6520 - loss: 1.1413
Epoch 20: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6617 - loss: 1.1218
Epoch 20: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6566 - loss: 1.0911
|

144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6080 - loss: 1.0193
| 17        | 0.652     | 4.835     | 550.3     | 0.08509   | 0.02851   | 80.28     | 2.108     | 2.076     | 2.175     | 0.4651    | 14.49     | 0.8947    | 3.933     |
Epoch 44: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6451 - loss: 35.9871
Epoch 20: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6552 - loss: 40.3579
Epoch 20: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1888 - loss: 52.7373
Epoch 21: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0367 - loss: 40.6170
Epoch 26: early stopping
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0757 - loss: 21.2429
| 18        | 0.3181    | 6.133     | 787.1     | 0.7578    | 0.2121    | 54.08     | 1.11      | 1.485     | 1.567     | 0.6181    | 35.6      | 0.824     | 6.439     |
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8087 - loss: 0.5285
Epoch

In [25]:
best_params = nn_opt.max['params']
best_score = nn_opt.max['target']

print(f"Best Parameters: {best_params}")
print(f"Highest Accuracy: {best_score}")

Best Parameters: {'activation': 2.79884089544096, 'batch_size': 460.14665762139765, 'dropout': 0.7296061783380641, 'dropout_rate': 0.19126724140656393, 'epochs': 90.97701940610612, 'kernel': 1.9444298503238986, 'layers1': 1.2391884918766034, 'layers2': 2.42648957444599, 'learning_rate': 0.7631771981307285, 'neurons': 60.51494778125466, 'normalization': 0.770967179954561, 'optimizer': 3.456569174550735}
Highest Accuracy: 0.8231808304786682


In [26]:
# Retrieve the best parameters from the optimization result
optimum = nn_opt.max['params']

# Assign the best parameters to their respective variables
learning_rate = optimum['learning_rate']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', 'LeakyReLU', 'relu']
activation = activationL[round(optimum['activation'])]

# Convert the hyperparameters to their integer form where necessary
optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])
optimum['kernel'] = round(optimum['kernel'])

optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
optimizerD = {
    'SGD': SGD(learning_rate=learning_rate),
    'Adam': Adam(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}

# Retrieve the optimizer name
optimizer_name = optimizerL[round(optimum['optimizer'])]

# Print the optimum parameters in a readable format
print(f"Best Parameters: ")
print(f"Activation: {activation}")
print(f"Batch Size: {optimum['batch_size']}")
print(f"Dropout Rate: {optimum['dropout_rate']:.4f}")
print(f"Epochs: {optimum['epochs']}")
print(f"Kernel Size: {optimum['kernel']}")
print(f"Layers1: {optimum['layers1']}")
print(f"Layers2: {optimum['layers2']}")
print(f"Learning Rate: {optimum['learning_rate']:.4f}")
print(f"Neurons: {optimum['neurons']}")
print(f"Normalization: {optimum['normalization']:.4f}")
print(f"Optimizer: {optimizer_name}")

Best Parameters: 
Activation: softsign
Batch Size: 460
Dropout Rate: 0.1913
Epochs: 91
Kernel Size: 2
Layers1: 1
Layers2: 2
Learning Rate: 0.7632
Neurons: 61
Normalization: 0.7710
Optimizer: Adadelta


In [47]:
# Best parameters from optimization
best_params = {
    'neurons': 61,
    'kernel': 2,
    'activation': 'softsign',
    'optimizer': 'Adadelta',
    'learning_rate': 0.7632,
    'batch_size': 460,
    'epochs':91,
    'layers1': 1,
    'layers2': 2,
    'normalization': 0.7710,
    'dropout': 0.7296,
    'dropout_rate':  0.1913
}

# Initialize optimizer with learning rate
optimizers = {
    'Adam': Adam(learning_rate=best_params['learning_rate']),
    'SGD': SGD(learning_rate=best_params['learning_rate']),
    'RMSprop': RMSprop(learning_rate=best_params['learning_rate']),
    'Adadelta': Adadelta(learning_rate=best_params['learning_rate']),
    'Adagrad': Adagrad(learning_rate=best_params['learning_rate']),
    'Adamax': Adamax(learning_rate=best_params['learning_rate']),
    'Nadam': Nadam(learning_rate=best_params['learning_rate']),
    'Ftrl': Ftrl(learning_rate=best_params['learning_rate'])
}

optimizer = optimizers[best_params['optimizer']]

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
#n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(best_params['neurons'], kernel_size=best_params['kernel'], activation=best_params['activation'], input_shape=(15,9)))

if best_params['normalization'] > 0.5:
    model.add(BatchNormalization())

for _ in range(best_params['layers1']):
    model.add(Dense(best_params['neurons'], activation=best_params['activation']))

if best_params['dropout'] > 0.5:
    model.add(Dropout(best_params['dropout_rate'], seed=123))

for _ in range(best_params['layers2']):
    model.add(Dense(best_params['neurons'], activation=best_params['activation']))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # sigmoid, tanh, softmax

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model with the optimized parameters
model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=2)


/Users/mariazaremba/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/91
40/40 - 12s - 305ms/step - accuracy: 0.6267 - loss: 1.2788
Epoch 2/91
40/40 - 8s - 191ms/step - accuracy: 0.6974 - loss: 0.8765
Epoch 3/91
40/40 - 6s - 150ms/step - accuracy: 0.7226 - loss: 0.7993
Epoch 4/91
40/40 - 7s - 170ms/step - accuracy: 0.7436 - loss: 0.7442
Epoch 5/91
40/40 - 7s - 183ms/step - accuracy: 0.7582 - loss: 0.7043
Epoch 6/91
40/40 - 4s - 102ms/step - accuracy: 0.7751 - loss: 0.6674
Epoch 7/91
40/40 - 4s - 95ms/step - accuracy: 0.7827 - loss: 0.6400
Epoch 8/91
40/40 - 5s - 113ms/step - accuracy: 0.7877 - loss: 0.6174
Epoch 9/91
40/40 - 4s - 94ms/step - accuracy: 0.7966 - loss: 0.5928
Epoch 10/91
40/40 - 4s - 96ms/step - accuracy: 0.8059 - loss: 0.5697
Epoch 11/91
40/40 - 4s - 107ms/step - accuracy: 0.8073 - loss: 0.5597
Epoch 12/91
40/40 - 4s - 96ms/step - accuracy: 0.8174 - loss: 0.5353
Epoch 13/91
40/40 - 4s - 93ms/step - accuracy: 0.8212 - loss: 0.5216
Epoch 14/91
40/40 - 4s - 94ms/step - accuracy: 0.8260 - loss: 0.5059
Epoch 15/91
40/40 - 4s - 93ms/step

In [48]:
model.summary()

Model: "sequential_96"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_96 (Conv1D)              │ (None, 14, 61)         │         1,159 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_41          │ (None, 14, 61)         │           244 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_479 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_60 (Dropout)            │ (None, 14, 61)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_480 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_481 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_96 (MaxPooling1D) │ (None, 7, 61)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_96 (Flatten)            │ (None, 427)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_482 (Dense)               │ (None, 15)             │         6,420 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 57,265 (223.70 KB)

 Trainable params: 19,047 (74.40 KB)

 Non-trainable params: 122 (488.00 B)

 Optimizer params: 38,096 (148.82 KB)

In [49]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [50]:
# Assuming X_train and y_train are your input data and labels
# One-hot encode y_train
y_train_one_hot = to_categorical(y_train, num_classes=15)

In [51]:
# Check shapes
print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

X_train shape: (18360, 15, 9)
y_train_one_hot shape: (18360, 15)


In [52]:
model.fit(X_train, y_train_one_hot, batch_size=460, epochs=91, verbose=2)

Epoch 1/91
40/40 - 9s - 226ms/step - accuracy: 0.9509 - loss: 0.1434
Epoch 2/91
40/40 - 3s - 84ms/step - accuracy: 0.9533 - loss: 0.1346
Epoch 3/91
40/40 - 3s - 72ms/step - accuracy: 0.9562 - loss: 0.1260
Epoch 4/91
40/40 - 3s - 72ms/step - accuracy: 0.9587 - loss: 0.1171
Epoch 5/91
40/40 - 3s - 70ms/step - accuracy: 0.9613 - loss: 0.1145
Epoch 6/91
40/40 - 3s - 85ms/step - accuracy: 0.9586 - loss: 0.1169
Epoch 7/91
40/40 - 3s - 79ms/step - accuracy: 0.9557 - loss: 0.1278
Epoch 8/91
40/40 - 3s - 76ms/step - accuracy: 0.9599 - loss: 0.1145
Epoch 9/91
40/40 - 3s - 76ms/step - accuracy: 0.9587 - loss: 0.1180
Epoch 10/91
40/40 - 3s - 75ms/step - accuracy: 0.9635 - loss: 0.1089
Epoch 11/91
40/40 - 3s - 81ms/step - accuracy: 0.9649 - loss: 0.1027
Epoch 12/91
40/40 - 3s - 85ms/step - accuracy: 0.9641 - loss: 0.1050
Epoch 13/91
40/40 - 3s - 75ms/step - accuracy: 0.9666 - loss: 0.1007
Epoch 14/91
40/40 - 3s - 77ms/step - accuracy: 0.9657 - loss: 0.0977
Epoch 15/91
40/40 - 3s - 75ms/step - accur

In [53]:
# Change this to Weather true/false
stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [54]:
print("Unique classes in y_test:", np.unique(y_test))

Unique classes in y_test: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14]


In [55]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])



In [56]:
# Before making predictions, convert y_test to one-hot format
y_test_one_hot = to_categorical(y_test, num_classes=15)

In [57]:
y_pred = model.predict(X_test)

144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [58]:
# Manually calculate accuracy
correct_predictions = np.sum(y_test_labels == y_pred_labels)
total_samples = len(y_test_labels)
accuracy = correct_predictions / total_samples

print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 95.84%


In [59]:
y_pred_labels = np.argmax(y_pred, axis=1)

In [60]:
# Convert y_test_one_hot back to class labels
y_test_labels = np.argmax(y_test_one_hot, axis=1)

In [61]:
cm = confusion_matrix(y_test, y_pred, stations)
print(cm)

Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        2862        25         3       1           2         2       0   
BELGRADE       34       903         6       1           0         1       0   
BUDAPEST        9         1       149       6           0         0       0   
DEBILT          8         1         1      61           2         0       1   
DUSSELDORF      6         0         1       4          17         3       0   
HEATHROW        3         0         1       1           1        52       1   
KASSEL          0         0         0       0           0         0       8   
LJUBLJANA       2         0         0       0           0         0       0   
MAASTRICHT      0         0         0       1           1         0       0   
MADRID          9         0         2       0           0         0       0   
MUNCHENB        0         0         0       0       